In [ ]:
import os
import sys
project_root_dir = os.path.join(os.getcwd(),'../..')
if project_root_dir not in sys.path:
    sys.path.append(project_root_dir)

import config
import numpy as np
import torch

from utils import show_abundance, plot_endmembers
from dataset import Samson
from HySpecLab.metrics import rmse, sad
from matplotlib import pyplot as plt
from scipy import io

In [ ]:
from dataset import Samson
dataset = Samson(config.Samson_PATH)
result_path = os.path.join(config.RESULTS_PATH, 'samson')

## Results

In [ ]:
idnet_result_path = os.path.join(result_path, 'idnet/result')
endmember_init = ['nfindr', 'vca']

M = dataset.endmembers()
A = dataset.abundance()
Y = (dataset.A @ dataset.endmembers()).reshape(95, 95, -1)

for _endmember_init in endmember_init:
    files = os.listdir(os.path.join(idnet_result_path, f'{_endmember_init}'))
    batch_rmse_x = []
    batch_rmse_a = []
    batch_sad_m = []
    for f in files:
        if f.endswith('.mat'):
            result = io.loadmat(os.path.join(idnet_result_path, f'{_endmember_init}/{f}'))
            M_hat = torch.tensor(result['Mn_est']).mean(axis=2).T
            A_hat = torch.tensor(np.transpose(result['A_est'].reshape(3,95,95), (1,2,0)))
            X_hat = torch.tensor(np.transpose(result['Y_rec'].reshape(-1, 95, 95), (1,2,0)))

            rmse_x = rmse(Y, X_hat, dim=None)
            rmse_a = rmse(A, A_hat, dim=None)
            sad_x = np.diagonal(sad(M, M_hat)).mean()
            batch_rmse_x.append(rmse_x)
            batch_rmse_a.append(rmse_a)
            batch_sad_m.append(sad_x)
        
    import pandas as pd
    df = pd.DataFrame(columns=['RMSE_X', 'RMSE_A', 'SAD_M'])
    df['RMSE_X'] = batch_rmse_x
    df['RMSE_A'] = batch_rmse_a
    df['SAD_M'] = batch_sad_m

    # extract mean and std
    df.to_csv(os.path.join(idnet_result_path, 'metrics_{}_batch.csv'.format(_endmember_init)), index=False)
    print(_endmember_init, df['RMSE_X'].mean(), df['RMSE_X'].std(), df['RMSE_A'].mean(), df['RMSE_A'].std(), df['SAD_M'].mean(), df['SAD_M'].std())

In [ ]:
from utils import show_abundance

idnet_imgs_path = os.path.join(result_path, 'idnet/imgs')

fig = show_abundance(A_hat, transpose=False)
plt.show(fig)

fig.savefig(os.path.join(idnet_imgs_path, 'A_hat.pdf'), bbox_inches='tight')